In [12]:
# dynamicznie | aka wspolczynniki postaci Newtona
function difference_quotients(f,args)
    values = map(f,args)
    
    for i in 1:length(args)-1
        for j in length(args):-1:1+i
            values[j] = ( values[j] - values[j-1] ) / ( args[j] - args[j-i] )
#             @show values
        end
#         @show values
    end
    
    return values; 
end 

# s(x) = x^4 - x^3 + x^2 -x + 1
# @show difference_quotients(s,[-2,-1,0,1,2,3])


difference_quotients (generic function with 1 method)

In [13]:
function lambda(k,args)
    return h(k,args)/(h(k+1,args) + h(k,args))
end

function h(k,args)
    return args[k] - args[k-1]
end

function d(k,f,args)
    return 6*difference_quotients(f,[args[k-1],args[k],args[k+1]])[3]
end

function Ms(f,args)
    n = length(args)
    qs = [0.0]
    us = [0.0]
    q = 0
    u = 0
    for k in 1:1:n-2
        p = lambda(k+1,args)*q + 2
        q = ( lambda(k+1,args)-1 ) / p
        u = ( d(k+1,f,args) - lambda(k+1,args)*u) / p
        push!(qs,q)
        push!(us,u)
    end
       
    Ms = [u]
    M = u
    for k in n-3:-1:1
        M = us[k+1] + qs[k+1]*M
        push!(Ms,M)
    end
    push!(Ms,0.0)
    Ms = reverse(Ms)
    push!(Ms,0.0)
    return Ms
end

# @show tmp

function spline(Ms,f,args)
    func = [];
    for k in 1:1:length(args)-1
        g(x) = ( (Ms[k]*(args[k+1]-x)^3)/6 + (Ms[k+1]*(x-args[k])^3)/6 + (f(args[k])-(Ms[k]*(h(k+1,args)^2))/6)*(args[k+1] -x) +   (f(args[k+1])-(Ms[k+1]*(h(k+1,args)^2))/6)*(x-args[k]) )/h(k+1,args) 
        push!(func,g)
    end
    return func
end



# @show y[3](1)

4-element Array{Any,1}:
 g
 g
 g
 g

In [14]:
function spline_value(spline_array,args,x)
    if x == args[1]
        return spline_array[1](x)
    elseif x == args[length(args)]
        return spline_array[length(spline_array)](x)
    end
    nr = 0
    for j in args
        if j < x
            nr += 1
        end
    end
    return spline_array[nr](x)
end

spline_value (generic function with 1 method)

In [15]:
function spline_error(spline_array,f,args,N)
    a = args[1]
    b = args[length(args)]
    h = (b - a) / (length(N)-1)
    max = 0
    
    for i in a:h:b
        if i == a
            nr = 1
        elseif i == b
            nr = length(args)-1
        else
            nr = 0
            for j in args
                if j < i
                    nr += 1
                end
            end
        end
             
#         @show nr
        tmp = abs(spline_array[nr](i) - f(i))
        if tmp > max
            max = tmp
        end
    end
    
    return max
end

# @show spline_error(y,f,args,[-2,-1.5,-1,-0.5,0,0.5,1,1.5,2])

spline_error (generic function with 1 method)

In [19]:

using Plotly
function draw(f,args,spline_array)
    a = args[1]
    b = args[length(args)]
    
    _args = a:0.1:b
    func_values = []
    spline_values = []
    
    for i in _args
        push!(func_values,f(i))
        push!(spline_values,spline_value(spline_array,args,i))
    end 
    
    func = scatter(;x=_args, y=func_values, mode="lines", name = "func")
    spline = scatter(;x=_args, y=spline_values, mode="lines", name = "spline")
    
    layout = Layout(;title="Title",xaxis=attr(title="args"),yaxis=attr(title="values"))
    
    plot([func, spline ],layout)
end

f(x) = x^3 - 8*x^2 + x + 5
g(x) = sin(x)
h(x) = exp(x)
r(x) = 1/(x^2+1)
argsr = -5:1:5

args = [-2,-1,0,1,2]

tmp1 = Ms(f,args)
tmp2 = Ms(g,args)

tmpr = Ms(r,argsr)

y = spline(tmp1,f,args)
z = spline(tmp2,g,args)

runge = spline(tmpr,r,argsr)

# draw(f,args,y)
# draw(g,args,z)
draw(r,argsr,runge)